# INFOVIS 2021 - Vacunaciones Argentina 2021
## 1) Obtención de Datos

Fuente: Ministerio de Salud de la Nación, Dirección de Control de Enfermedades Inmunoprevenibles (DiCEI)

Dosis Aplicadas en la República Argentina - [Dataset](http://datos.salud.gob.ar/dataset/vacunas-contra-covid19-dosis-aplicadas-en-la-republica-argentina)

In [1]:
import pandas as pd
import os

In [ ]:
!wget -c https://sisa.msal.gov.ar/datos/descargas/covid-19/files/datos_nomivac_covid19.zip

In [ ]:
!unzip datos_nomivac_covid19.zip

In [2]:
df = pd.read_csv("datos_nomivac_covid19.csv", dtype= {'jurisdiccion_residencia_id' : object, 
                                                      'depto_residencia_id'        : object, 
                                                      'jurisdiccion_aplicacion_id' : object, 
                                                      'depto_aplicacion_id'        : object})

In [4]:
df.head()

,sexo,grupo_etario,jurisdiccion_residencia,jurisdiccion_residencia_id,depto_residencia,depto_residencia_id,jurisdiccion_aplicacion,jurisdiccion_aplicacion_id,depto_aplicacion,depto_aplicacion_id,fecha_aplicacion,vacuna,condicion_aplicacion,orden_dosis,lote_vacuna
0,M,30-39,Buenos Aires,06,La Plata,441,Buenos Aires,06,Quilmes,658,2021-07-25,Sinopharm,18 a 39 años SIN Factores de Riesgo,1,202106B1335
1,F,18-29,Corrientes,18,Capital,021,Corrientes,18,Capital,021,2021-07-23,Sinopharm,18 a 39 años SIN Factores de Riesgo,1,202106B1345
2,M,60-69,Misiones,54,Eldorado,042,Misiones,54,Eldorado,042,2021-06-23,Sputnik,18 a 59 años CON Factores de Riesgo,1,I-810421
3,F,70-79,Buenos Aires,06,General Pueyrredón,357,Buenos Aires,06,General Pueyrredón,357,2021-08-12,Moderna,60 o más años,2,078C21A
4,F,30-39,San Juan,70,Santa Lucía,098,San Juan,70,Capital,028,2021-08-19,Moderna,18 a 59 años CON Factores de Riesgo,2,022D21A


In [8]:
df_jurisdiccion_aplicacion.rename(columns = {'jurisdiccion_aplicacion' : 'jurisdiccion'}, inplace = True)

## 2) Jurisdicciones con vacunaciones coincidentes y no coincidentes con su jurisdicción de residencia 

Vacunaciones no coincidentes (no fueron vacunados en su jurisdicción de residencia)

In [25]:
df_vacunaciones_no_foraneas = df[df["jurisdiccion_residencia_id"] != df["jurisdiccion_aplicacion_id"]].groupby(by="jurisdiccion_aplicacion").size().to_frame(name="cantidad").reset_index()

In [26]:
df_vacunaciones_no_foraneas

,jurisdiccion_aplicacion,cantidad
0,Buenos Aires,834354
1,CABA,520471
2,Catamarca,11223
3,Chaco,44580
4,Chubut,26383
5,Corrientes,34689
6,Córdoba,89178
7,Entre Ríos,35236
8,Formosa,26652
9,Jujuy,29720


Vacunaciones coincidentes (fueron vacunados en su jurisdicción de residencia)

In [27]:
df_vacunaciones_foraneas = df[df["jurisdiccion_residencia_id"] == df["jurisdiccion_aplicacion_id"]].groupby(by="jurisdiccion_aplicacion").size().to_frame(name="cantidad").reset_index()

In [28]:
df_vacunaciones_foraneas

,jurisdiccion_aplicacion,cantidad
0,Buenos Aires,18923952
1,CABA,3684814
2,Catamarca,468198
3,Chaco,1111476
4,Chubut,604820
5,Corrientes,1103448
6,Córdoba,4328855
7,Entre Ríos,1446575
8,Formosa,679754
9,Jujuy,790802


Juntando datos de totales vacunados coincidentes y No coincidentes

In [29]:
df_merge_vacunaciones_foraneas = pd.merge(df_vacunaciones_foraneas, df_vacunaciones_no_foraneas, how='inner', on='jurisdiccion_aplicacion')
df_merge_vacunaciones_foraneas.rename(columns = {'cantidad_x' : 'cantidad_foraneas', 'cantidad_y' : 'cantidad_no_foraneas'}, inplace = True)

In [30]:
df_merge_vacunaciones_foraneas

,jurisdiccion_aplicacion,cantidad_foraneas,cantidad_no_foraneas
0,Buenos Aires,18923952,834354
1,CABA,3684814,520471
2,Catamarca,468198,11223
3,Chaco,1111476,44580
4,Chubut,604820,26383
5,Corrientes,1103448,34689
6,Córdoba,4328855,89178
7,Entre Ríos,1446575,35236
8,Formosa,679754,26652
9,Jujuy,790802,29720


Exportamos en csv para poder trabajar con los datos

In [31]:
df_merge_vacunaciones_foraneas.to_csv(r'vacunaciones_foraneas.csv', index = False)

Vacunaciones no coincidentes por dosis (fueron vacunados en su jurisdicción de residencia)

In [32]:
df_vacunaciones_no_foraneas_dosis = df[df["jurisdiccion_residencia_id"] != df["jurisdiccion_aplicacion_id"]].groupby(["jurisdiccion_aplicacion", "orden_dosis"]).size().to_frame(name="cantidad").reset_index()

In [33]:
df_vacunaciones_no_foraneas_dosis

,jurisdiccion_aplicacion,orden_dosis,cantidad
0,Buenos Aires,1,485784
1,Buenos Aires,2,348570
2,CABA,1,284478
3,CABA,2,235993
4,Catamarca,1,7247
5,Catamarca,2,3976
6,Chaco,1,27252
7,Chaco,2,17328
8,Chubut,1,16435
9,Chubut,2,9948


Vacunaciones coincidentes por dosis (fueron vacunados en su jurisdicción de residencia)

In [34]:
df_vacunaciones_foraneas_dosis = df[df["jurisdiccion_residencia_id"] == df["jurisdiccion_aplicacion_id"]].groupby(["jurisdiccion_aplicacion", "orden_dosis"]).size().to_frame(name="cantidad").reset_index()

In [35]:
df_vacunaciones_foraneas_dosis

,jurisdiccion_aplicacion,orden_dosis,cantidad
0,Buenos Aires,1,10924460
1,Buenos Aires,2,7999492
2,CABA,1,2026425
3,CABA,2,1658389
4,Catamarca,1,290355
5,Catamarca,2,177843
6,Chaco,1,682937
7,Chaco,2,428539
8,Chubut,1,357504
9,Chubut,2,247316


In [36]:
df_merge_vacunaciones_foraneas_dosis = pd.merge(df_vacunaciones_foraneas_dosis, df_vacunaciones_no_foraneas_dosis,how='left', left_on=['jurisdiccion_aplicacion','orden_dosis'], right_on = ['jurisdiccion_aplicacion','orden_dosis'])
df_merge_vacunaciones_foraneas_dosis.rename(columns = {'cantidad_x' : 'cantidad_foraneas', 'cantidad_y' : 'cantidad_no_foraneas', 'orden_dosis_x' : 'orden_dosis'}, inplace = True)

In [37]:
df_merge_vacunaciones_foraneas_dosis

,jurisdiccion_aplicacion,orden_dosis,cantidad_foraneas,cantidad_no_foraneas
0,Buenos Aires,1,10924460,485784
1,Buenos Aires,2,7999492,348570
2,CABA,1,2026425,284478
3,CABA,2,1658389,235993
4,Catamarca,1,290355,7247
5,Catamarca,2,177843,3976
6,Chaco,1,682937,27252
7,Chaco,2,428539,17328
8,Chubut,1,357504,16435
9,Chubut,2,247316,9948


Exportamos en csv para poder trabajar con los datos

In [38]:
df_merge_vacunaciones_foraneas_dosis.to_csv(r'vacunaciones_foraneas_dosis.csv', index = False)